Dataset Credits: https://www.kaggle.com/datasets/saurabhshahane/ecommerce-text-classification
We have a dataset of ecommerce item description. Total 4 categories,

Household
Electronics
Clothing and Accessories
Books
The task at hand is to classify a product into one of the above 4 categories based on the product description

In [36]:
import pandas as pd

df= pd.read_csv("dataset/ecommerce_dataset.csv", names=["category", "description"], header=None)
print(df.shape)
df.head(3)

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...


In [37]:
df.category.value_counts()

category
Household                 19313
Books                     11820
Electronics               10621
Clothing & Accessories     8671
Name: count, dtype: int64

In [38]:

df.dropna(inplace=True)
df.shape

(50424, 2)

In [39]:
df.category.unique()


array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [40]:
df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)


In [41]:
df.category

0          Household
1          Household
2          Household
3          Household
4          Household
            ...     
50420    Electronics
50421    Electronics
50422    Electronics
50423    Electronics
50424    Electronics
Name: category, Length: 50424, dtype: object

In [42]:
df['category'] = "__label__" + df['category'].astype(str)

df.head(5)

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [43]:
df['category_description'] =df['category']+" "+df['description']

In [44]:
df.head(5)

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__Household SAF Flower Print Framed Pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__Household Incredible Gifts India Wood...


In [45]:
df['category_description'][0]

'__label__Household Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and so

Pre-procesing


        Remove punctuation

        Remove extra space

        Make the entire sentence lower case

In [46]:
import re
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower() 

In [47]:

df['category_description'] = df['category_description'].map(preprocess)
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


# Train Test Split



In [48]:
from sklearn.model_selection import train_test_split

train , test = train_test_split(df , test_size=0.2)

In [49]:
train.shape, test.shape


((40339, 3), (10085, 3))

In [50]:
train.to_csv('ecommerce.train' ,columns=["category_description"], index=False, header=False)
test.to_csv('ecommerce.test' ,columns=["category_description"], index=False, header=False)


# Train the model and evaluate performance



In [51]:
import fasttext

model = fasttext.train_supervised(input="ecommerce.train")


Read 4M words
Number of words:  79137
Number of labels: 4
Progress: 100.0% words/sec/thread: 3123946 lr:  0.000000 avg.loss:  0.172423 ETA:   0h 0m 0s


In [52]:
model.test("ecommerce.test")

(10085, 0.969261279127417, 0.969261279127417)

# Now let's do prediction for few product descriptions



In [53]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")


(('__label__electronics',), array([0.99704635]))

In [54]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")


(('__label__clothing_accessories',), array([1.00001001]))

In [55]:
model.predict("think and grow rich deluxe edition")


(('__label__books',), array([1.00000989]))

In [56]:
model.get_nearest_neighbors("painting")


[(0.9970274567604065, 'security'),
 (0.9963123798370361, 'steam'),
 (0.9960723519325256, 'inverter'),
 (0.9957046508789062, 'vegetables'),
 (0.9956219792366028, 'primer'),
 (0.9953204393386841, 'shower'),
 (0.9948866367340088, 'guard'),
 (0.9947380423545837, 'stove'),
 (0.9947187900543213, 'covers'),
 (0.9946399927139282, 'lint')]

In [57]:
model.get_nearest_neighbors("sony")


[(0.9985144734382629, 'devices'),
 (0.9980754852294922, 'ceiling'),
 (0.9977045059204102, 'point'),
 (0.9974853992462158, 'stereo'),
 (0.9974591135978699, 'correction'),
 (0.9972898960113525, 'gps'),
 (0.996933102607727, 'dvd'),
 (0.9968894720077515, 'binoculars'),
 (0.9965425133705139, 'external'),
 (0.9957712292671204, 'calculator')]

In [58]:
model.get_nearest_neighbors("banglore")


[(0.0, 'to'),
 (0.0, 'and'),
 (0.0, 'a'),
 (0.0, 'with'),
 (0.0, 'for'),
 (0.0, 'is'),
 (0.0, '</s>'),
 (0.0, 'pgdmle'),
 (0.0, 'fnams'),
 (0.0, 'milann')]